In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Crypto_currency_data/bitcoin_final_data.csv")
df.head()

,Date,Close(BTC-USD),High(BTC-USD),Low(BTC-USD),Open(BTC-USD),Volume(BTC-USD),Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,17-09-2014,457.334015,468.174011,452.421997,465.864014,21056800.0,NaN,NaN,NaN,NaN
1,18-09-2014,424.440002,456.859985,413.104004,456.859985,34483200.0,NaN,NaN,NaN,NaN
2,19-09-2014,394.795990,427.834992,384.532013,424.102997,37919700.0,NaN,NaN,NaN,NaN
3,20-09-2014,408.903992,423.295990,389.882996,394.673004,36863600.0,NaN,NaN,NaN,NaN
4,21-09-2014,398.821014,412.425995,393.181000,408.084992,26580100.0,NaN,NaN,NaN,NaN


In [ ]:
# Removing column contained unknown
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,Date,Close(BTC-USD),High(BTC-USD),Low(BTC-USD),Open(BTC-USD),Volume(BTC-USD)
0,17-09-2014,457.334015,468.174011,452.421997,465.864014,21056800.0
1,18-09-2014,424.440002,456.859985,413.104004,456.859985,34483200.0
2,19-09-2014,394.795990,427.834992,384.532013,424.102997,37919700.0
3,20-09-2014,408.903992,423.295990,389.882996,394.673004,36863600.0
4,21-09-2014,398.821014,412.425995,393.181000,408.084992,26580100.0


In [ ]:
closedf = df[['Date','Close(BTC-USD)']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (3771, 2)


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is your original DataFrame
closedf = df[['Date', 'Close(BTC-USD)']]

# Convert the 'Date' column from 'DD-MM-YYYY' format to datetime
closedf['Date'] = pd.to_datetime(closedf['Date'], format='%d-%m-%Y')

# Create a Plotly figure
fig = go.Figure()

# Add the closing price data
fig.add_trace(go.Scatter(
    x=closedf['Date'],  # Now the 'Date' column is in datetime format
    y=closedf['Close(BTC-USD)'],  # Plot the closing price
    mode='lines',
    name='Closing Price'
))

# Update layout for better visualization
fig.update_layout(
    title='Bitcoin Historical Closing Price',
    xaxis_title='Date',
    yaxis_title='Close Price (BTC-USD)',
    template='plotly_dark',  # Use a dark theme (optional)
    xaxis_rangeslider_visible=True,  # Show range slider for interactive zoom
)

# Show the plot
fig.show()

# Print the shape of the close dataframe
print("Shape of close dataframe:", closedf.shape)

<ipython-input-6-9ebb5d572975>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  closedf['Date'] = pd.to_datetime(closedf['Date'], format='%d-%m-%Y')


Shape of close dataframe: (3771, 2)


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'df' is your original DataFrame
closedf = df[['Date', 'Close(BTC-USD)']]

# Convert the 'Date' column from 'DD-MM-YYYY' format to datetime
closedf['Date'] = pd.to_datetime(closedf['Date'], format='%d-%m-%Y')

# Filter the data for dates from January 2024 onwards
closedf = closedf[closedf['Date'] >= '2024-01-01']

# Create a copy of the filtered DataFrame
close_stock = closedf.copy()

# Print the total number of data points for prediction
print("Total data for prediction: ", closedf.shape[0])

# Create a Plotly figure
fig = go.Figure()

# Add the closing price data
fig.add_trace(go.Scatter(
    x=closedf['Date'],  # Now the 'Date' column is in datetime format
    y=closedf['Close(BTC-USD)'],  # Plot the closing price
    mode='lines',
    name='Closing Price'
))

# Update layout for better visualization
fig.update_layout(
    title='Bitcoin Historical Closing Price',
    xaxis_title='Date',
    yaxis_title='Close Price (BTC-USD)',
    template='plotly_dark',  # Use a dark theme (optional)
    xaxis_rangeslider_visible=True,  # Show range slider for interactive zoom
)

# Show the plot
fig.show()

Total data for prediction:  378


<ipython-input-7-15aa27ada601>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

if 'Date' in closedf.columns:
    del closedf['Date']

scaler = MinMaxScaler(feature_range=(0, 1))
closedf_scaled = scaler.fit_transform(closedf[['Close(BTC-USD)']])
print(closedf_scaled.shape)

(378, 1)


In [ ]:
training_size = int(len(closedf_scaled) * 0.60)
train_data = closedf_scaled[0:training_size]
test_data = closedf_scaled[training_size:]

print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (226, 1)
test_data:  (152, 1)


In [ ]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (210, 15)
y_train:  (210,)
X_test:  (136, 15)
y_test (136,)


In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (210, 15, 1)
X_test:  (136, 15, 1)


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(10, input_shape=(None, 1), activation="relu"))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.0848 - val_loss: 0.2790
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0743 - val_loss: 0.2525
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0642 - val_loss: 0.2274
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0560 - val_loss: 0.2022
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0434 - val_loss: 0.1748
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0296 - val_loss: 0.1379
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0201 - val_loss: 0.1006
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0119 - val_loss: 0.0688
Epoch 9/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0073 - val_loss: 0.0462
Epoch 10/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0049 - val_loss: 0.0337
Epoch 11/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0046 - val_loss: 0.0285
Epoch 12/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0040 - val_lo

In [ ]:
import plotly.graph_objects as go

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(epochs),
    y=loss,
    mode='lines',
    name='Training Loss',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=list(epochs),
    y=val_loss,
    mode='lines',
    name='Validation Loss',
    line=dict(color='blue')
))

fig.update_layout(
    title='Training and Validation Loss',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    legend=dict(x=0, y=1, traceorder='normal'),
    template='plotly_white'
)

fig.show()

In [ ]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


((210, 1), (136, 1))

In [ ]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

In [ ]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Assuming original_ytrain, train_predict, original_ytest, and test_predict are defined
print("Train data RMSE:", math.sqrt(mean_squared_error(original_ytrain, train_predict)))
print("Train data MSE:", mean_squared_error(original_ytrain, train_predict))
print("Train data MAE:", mean_absolute_error(original_ytrain, train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE:", math.sqrt(mean_squared_error(original_ytest, test_predict)))
print("Test data MSE:", mean_squared_error(original_ytest, test_predict))
print("Test data MAE:", mean_absolute_error(original_ytest, test_predict))

Train data RMSE: 1786.0663396707191
Train data MSE: 3190032.9697047607
Train data MAE: 1311.6622949821422
-------------------------------------------------------------------------------------
Test data RMSE: 5428.595899417587
Test data MSE: 29469653.439173445
Test data MAE: 4082.1890525275726


In [ ]:
from sklearn.metrics import explained_variance_score

# Assuming original_ytrain, train_predict, original_ytest, and test_predict are defined
print("Train data explained variance regression score:",
      explained_variance_score(original_ytrain, train_predict))

print("Test data explained variance regression score:",
      explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9576017055326288
Test data explained variance regression score: 0.9386698509229525


In [ ]:
from sklearn.metrics import r2_score

# Assuming original_ytrain, train_predict, original_ytest, and test_predict are defined
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9575496785547571
Test data R2 score: 0.8950223611163683


In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from itertools import cycle

# Assuming 'closedf', 'train_predict', 'test_predict', 'close_stock', and 'time_step' are defined
look_back = time_step

# Prepare train predictions for plotting
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict) + look_back, :] = train_predict

# Prepare test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict) + (look_back * 2) + 1:len(closedf) - 1, :] = test_predict

# Create a DataFrame for plotting
plotdf = pd.DataFrame({
    'date': close_stock['Date'],
    'original_close': close_stock['Close(BTC-USD)'],
    'train_predicted_close': trainPredictPlot.reshape(1, -1)[0].tolist(),
    'test_predicted_close': testPredictPlot.reshape(1, -1)[0].tolist()
})

# Plot using Plotly Express
fig = px.line(
    plotdf,
    x='date',
    y=['original_close', 'train_predicted_close', 'test_predicted_close'],
    labels={'value': 'Stock price', 'date': 'Date'}
)

# Update layout
fig.update_layout(
    title_text='Comparison between Original Close Price vs Predicted Close Price',
    plot_bgcolor='white',
    font_size=15,
    font_color='black',
    legend_title_text='Close Price'
)

# Update trace names
names = cycle(['Original close price', 'Train predicted close price', 'Test predicted close price'])
fig.for_each_trace(lambda t: t.update(name=next(names)))

# Update axes
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objects as go

# Define the accuracy
accuracy = 89.5022

# Create a gauge chart
fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=accuracy,
    title={'text': "Model Accuracy"},
    gauge={
        'axis': {'range': [0, 100]},
        'bar': {'color': "darkblue"},
        'steps': [
            {'range': [0, 50], 'color': "lightgray"},
            {'range': [50, 75], 'color': "gray"},
            {'range': [75, 100], 'color': "lightblue"}
        ],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': accuracy
        }
    }
))

# Update layout
fig.update_layout(
    title="Model Accuracy Gauge",
    font=dict(size=18)
)

# Show the plot
fig.show()